In [1]:
import os
import sys 
from flask import Flask
from flask import request
from flask_api import status
import psycopg2
import json
import copy
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))
datosBaseDeDatos = {"usuario":"postgres","password":"carlos98","Host":"127.0.0.1","puerto":"5432","DB":"chatblocks"}

In [2]:
def abrirJson(name,path=os.getcwd()+"/" ):
    with open(path+name,encoding='utf-8') as data_file:  
        diccionario = json.load(data_file)
    return diccionario

In [3]:
def conexionBaseDeDatos(datosConexion):
    try:
        connection = psycopg2.connect(user = datosConexion["usuario"],
                              password = datosConexion["password"],
                              host = datosConexion["Host"],
                              port = datosConexion["puerto"],
                              database = datosConexion["DB"])    
        return connection
    except(Exception, psycopg2.Error) as error:
        print("Error al conectar con base de datos", error)

In [4]:
def consultar(connection,datos=False,consulta=False,tabla=False):
    #print("datos",datos)
    consultaBots = list()
    column_names = []
    data_rows = []
    diccionarioDatos = dict()
    consultas = list()
    cursor = connection.cursor()  
    cosulta =""    
    if datos:
        print("intenatando consulta...")
        if tabla == 'robots':
            consulta = "select * from robots where id_face = '"+datos["id_face"]+"';"
        else:
            consulta = "select * from "+tabla+" where "        
            i=0
            for dato, valorDato in datos.items():
                if i==0:
                    consulta = consulta+dato+"= '"+str(valorDato)+"'"                
                else:             
                    consulta = consulta+" and "+dato+"= '"+str(valorDato)+"'" 
                i=i+1
            #print(dato,":",valorDato)
            consulta= consulta + " ;" 
            #print(consulta)        
            #consulta = "select * from robots where name_robot = '"+datosBot["name_robot"]+"' and id_face = '"+ datosBot["id_face"]+"' and  type_blocki = '"+datosBot["type_blocki"]+"' and access_token= '"+datosBot["access_token"]+"' and id_user = '"+datosBot["id_user"]+"';"
    print("consulta:",consulta)
    cursor.execute(consulta)    
    column_names = [desc[0] for desc in cursor.description]  
    rows = cursor.fetchall()
    for row in rows:
        diccionarioDatos = dict()
        for i in range(len(column_names)):
            diccionarioDatos.update({column_names[i]:row[i]})
        consultaBots.append(diccionarioDatos)
    cursor.close()
    connection.close()  
    
    if len(consultaBots)==1:
        return consultaBots[0]
    elif len(consultaBots)==0:
        print('La consulta no trajo nada')
        return {"error": "true"}
    else:
        return consultaBots

In [5]:
def insert(datos,tabla,connection):
    cursor = connection.cursor()
    values = ""
    datosInsert = ""
    i = 0;
    for dato, valorDato in datos.items():
        if i==0:
            values = values+dato 
            datosInsert = datosInsert+"'"+str(valorDato)+"' "                
        else:
            values = values+","+dato
            datosInsert =datosInsert+",'"+str(valorDato)+"' "
        i=i+1
    insersion  = "insert into "+tabla+" ( "+values+" ) values ("+datosInsert+" );" 
    cursor.execute(insersion)
    connection.commit()
    cursor.close()
    connection.close()
    return datos

In [6]:
def update(datosActualizar,datos,tabla,connection):
    actualizar =" "
    actualizar = "update "+tabla+" set"
    j = 0
    for dato,informacionDato in datosActualizar.items():
        print(dato)
        if j==0:
            actualizar=actualizar+" "+dato+"="+str(informacionDato)
            j=j+1
        else:
             actualizar=actualizar+", "+dato+"="+str(informacionDato)
    
    i=0
    actualizar=actualizar+" where "
    for dato, valorDato in datos.items():
        if i==0:
            actualizar = actualizar+dato+"= '"+str(valorDato)+"'"                
        else:             
            actualizar = actualizar+" and "+dato+"= '"+str(valorDato)+"'" 
        i=i+1
    actualizar= actualizar + " ;"  

In [7]:
app = Flask(__name__)
@app.route('/access_token', methods = ['POST'])
def postAccessToken():
    try:         
        connection = conexionBaseDeDatos(datosBaseDeDatos)        
        datosVariable = request.get_json()   
        tabla = "robots"
        con = consultaBots(connection,datos=datosVariable,tabla=tabla)        
        listaAcessToken=list()
        if type(con)==list:
            for i in range(len(con)):
                print(con[i])
                listaAcessToken.append(con[i]["access_token"])
        else:
            listaAcessToken.append(con["access_token"])         
        print(listaAcessToken)
        return {"access_token":listaAcessToken}
    except:
        return datosVariable,status.HTTP_404_NOT_FOUND


In [8]:
@app.route('/insert/<tabla>',methods=['POST'])
def funcionalidad(tabla):
    try:
        connection = conexionBaseDeDatos(datosBaseDeDatos)
        datos = request.get_json()
        print("Datos recibidos INSERT:")
        print(datos)
        insert(datos,str(tabla),connection)
        connection = conexionBaseDeDatos(datosBaseDeDatos)
        consulta = consultar(connection,datos=datos,tabla=str(tabla))
        print("Datos enviados INSERT:")
        print (consulta)
        return json.dumps(consulta)
    except (Exception, psycopg2.Error) as error :
        print("error",error)
        return tabla,status.HTTP_404_NOT_FOUND

In [9]:
@app.route('/select/<tabla>',methods=['GET'])
def select(tabla):
    try:
        connection = conexionBaseDeDatos(datosBaseDeDatos)
        datos = request.get_json()
        print("Datos recibidos SELECT:")
        print(datos)
        
        consulta = consultar(connection,datos=datos,tabla=str(tabla))
        print("Datos enviados SELECT:")
        print (consulta)
        return json.dumps(consulta)
    except (Exception, psycopg2.Error) as error :
        print("error",error)
        return tabla,status.HTTP_404_NOT_FOUND

In [ ]:
if __name__ == "__main__":    
    app.run(host='localhost',port='8181')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:8181/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Jun/2021 23:50:34] "GET /select/robots HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2021 23:50:34] "GET /select/bloqueslide HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2021 23:50:34] "GET /select/elementos HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2021 23:50:34] "GET /select/elementos HTTP/1.1" 200 -


Datos recibidos SELECT:
{'name_robot': 'Chatbot TT 2021', 'id_face': '110373671266869', 'block_ini': 'Informativo | a0be25bb-4eb4-49c2-830f-6eb23d64966c | ZG7udwy8rzhE5xtUnOGl', 'type_blocki': 'informativo', 'access_token': 'EAALZCCZBzPwfkBAAtF3N1x3GCvg9UuN7alfdTNGmZC5d96EY7vYBM6hQQedeMtNTZAWxFVFBmRXfT2lp3m970ZCDZBhelEyxvK6PN9gHzEXZBbdxvqXqETQzUffJynrycHPnBBR5GtxJl2ulvP0uXrP0fTyoEmQ5GchMN0ckdunXK6MEd1Q40Cb', 'id_user': 'charlygoldcruz@gmail.com', 'api_nlp': 'http://localhost:8282/'}
intenatando consulta...
consulta: select * from robots where id_face = '110373671266869';
Datos enviados SELECT:
{'name_robot': 'Chatbot TT 2021', 'id_face': '110373671266869', 'block_ini': 'Informativo | a0be25bb-4eb4-49c2-830f-6eb23d64966c | ZG7udwy8rzhE5xtUnOGl', 'type_blocki': 'informativo', 'access_token': 'EAALZCCZBzPwfkBAAtF3N1x3GCvg9UuN7alfdTNGmZC5d96EY7vYBM6hQQedeMtNTZAWxFVFBmRXfT2lp3m970ZCDZBhelEyxvK6PN9gHzEXZBbdxvqXqETQzUffJynrycHPnBBR5GtxJl2ulvP0uXrP0fTyoEmQ5GchMN0ckdunXK6MEd1Q40Cb', 'api_nlp': 

127.0.0.1 - - [14/Jun/2021 23:50:34] "POST /insert/variables HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2021 23:50:34] "POST /insert/bloqueslide HTTP/1.1" 200 -


Datos recibidos INSERT:
{'name_var': 'variableCarrCarrusel_|_5aaf3158-308e-4b01-9283-f01f32d587c94237', 'opc_type': 'Variable', 'var': 0, 'opc_data': 'String', 'id_robot': 538}
intenatando consulta...
consulta: select * from variables where name_var= 'variableCarrCarrusel_|_5aaf3158-308e-4b01-9283-f01f32d587c94237' and opc_type= 'Variable' and var= '0' and opc_data= 'String' and id_robot= '538' ;
Datos enviados INSERT:
{'id_var': 1816, 'opc_type': 'Variable', 'opc_data': 'String', 'var': '0', 'id_robot': 538, 'name_var': 'variableCarrCarrusel_|_5aaf3158-308e-4b01-9283-f01f32d587c94237'}
Datos recibidos INSERT:
{'namestate': 'Carrusel | 5aaf3158-308e-4b01-9283-f01f32d587c9 | 96628859-0899-7537-4503-489244348545 | ZG7udwy8rzhE5xtUnOGl', 'blocktype': 'slide', 'typingtime': '1', 'default_id': 'Carrusel | 5aaf3158-308e-4b01-9283-f01f32d587c9 | 96628859-0899-7537-4503-489244348545 | ZG7udwy8rzhE5xtUnOGl', 'id_var': 1816, 'id_robot': 538}
intenatando consulta...
consulta: select * from bloque

...

Datos enviados INSERT:
{'namestate': 'QuickReply | 9cd75eb1-5f87-4a23-84aa-d6de05ed56b8 | 07418629-1549-9289-4157-116148457825 | bSkwpaFVCmMXS8G2PR6k', 'contenido': 'QUICKREPLY 1', 'opciones': 'SALIDA 1', 'next_id': 'Input | cd6d4be8-c346-4550-a26e-4bb9a7230d3b | bSkwpaFVCmMXS8G2PR6k', 'blocktype': 'quickReply', 'typingtime': '1', 'default_id': 'QuickReply | 9cd75eb1-5f87-4a23-84aa-d6de05ed56b8 | 07418629-1549-9289-4157-116148457825 | bSkwpaFVCmMXS8G2PR6k', 'id_var': '1914', 'id_block': 692, 'id_robot': 540, 'opc_nextid': None, 'pos_y': None, 'pos_x': None, 'tag_active': None}
Datos recibidos INSERT:
{'name_var': 'variableQrQuickReply_|_f962f2fb-1047-4062-bac8-798fddc79ff23327', 'opc_type': 'Variable', 'var': 0, 'opc_data': 'String', 'id_robot': 540}
intenatando consulta...
consulta: select * from variables where name_var= 'variableQrQuickReply_|_f962f2fb-1047-4062-bac8-798fddc79ff23327' and opc_type= 'Variable' and var= '0' and opc_data= 'String' and id_robot= '540' ;
Datos enviados I

127.0.0.1 - - [15/Jun/2021 00:10:41] "POST /insert/bloqueinformativo HTTP/1.1" 200 -
127.0.0.1 - - [15/Jun/2021 00:10:41] "POST /insert/bloqueinformativo HTTP/1.1" 200 -


Datos recibidos INSERT:
{'next_id': 'Informativo | 98a8f39d-6761-48e7-aca7-66279cfabf40 | 07418629-1549-9289-4157-116148457825 | bSkwpaFVCmMXS8G2PR6k', 'namestate': 'Informativo | d9897d01-e0ce-44bd-ba3d-22973a7d74fd | bSkwpaFVCmMXS8G2PR6k', 'contenido': 'Soy el chatbot del TT-A056', 'contenttype': 'text', 'typingtime': '1', 'id_robot': 540, 'blocktype': 'informativo'}
intenatando consulta...
consulta: select * from bloqueinformativo where next_id= 'Informativo | 98a8f39d-6761-48e7-aca7-66279cfabf40 | 07418629-1549-9289-4157-116148457825 | bSkwpaFVCmMXS8G2PR6k' and namestate= 'Informativo | d9897d01-e0ce-44bd-ba3d-22973a7d74fd | bSkwpaFVCmMXS8G2PR6k' and contenido= 'Soy el chatbot del TT-A056' and contenttype= 'text' and typingtime= '1' and id_robot= '540' and blocktype= 'informativo' ;
Datos enviados INSERT:
{'namestate': 'Informativo | d9897d01-e0ce-44bd-ba3d-22973a7d74fd | bSkwpaFVCmMXS8G2PR6k', 'contenido': 'Soy el chatbot del TT-A056', 'next_id': 'Informativo | 98a8f39d-6761-48e7-

127.0.0.1 - - [15/Jun/2021 00:10:41] "POST /insert/variables HTTP/1.1" 200 -
127.0.0.1 - - [15/Jun/2021 00:10:41] "POST /insert/bloqueinput HTTP/1.1" 200 -
127.0.0.1 - - [15/Jun/2021 00:10:41] "POST /insert/bloqueinformativo HTTP/1.1" 200 -


Datos enviados INSERT:
{'id_var': 1916, 'opc_type': 'Variable', 'opc_data': 'String', 'var': '0', 'id_robot': 540, 'name_var': 'variableInputInput_|_cd6d4be8-c346-4550-a26e-4bb9a7230d3b6724'}
Datos recibidos INSERT:
{'next_id': 'Informativo | 34fa0024-002a-4673-8b70-6419afef7d4d | 19627070-4044-4254-7133-869533412022 | bSkwpaFVCmMXS8G2PR6k', 'default_id': 'Input | cd6d4be8-c346-4550-a26e-4bb9a7230d3b | bSkwpaFVCmMXS8G2PR6k', 'namestate': 'Input | cd6d4be8-c346-4550-a26e-4bb9a7230d3b | bSkwpaFVCmMXS8G2PR6k', 'contenido': 'Ingresa tu nombre', 'typingtime': '1', 'validacion': '', 'id_robot': 540, 'id_var': 1916, 'blocktype': 'input'}
intenatando consulta...
consulta: select * from bloqueinput where next_id= 'Informativo | 34fa0024-002a-4673-8b70-6419afef7d4d | 19627070-4044-4254-7133-869533412022 | bSkwpaFVCmMXS8G2PR6k' and default_id= 'Input | cd6d4be8-c346-4550-a26e-4bb9a7230d3b | bSkwpaFVCmMXS8G2PR6k' and namestate= 'Input | cd6d4be8-c346-4550-a26e-4bb9a7230d3b | bSkwpaFVCmMXS8G2PR6k' 